In [0]:
import dlt


read data

In [0]:
@dlt.table(
    table_properties = {"quality": "bronze"},
    name = "claim_bronze"
)
def claim_bronze():
    df = spark.readStream.table("dev.bronze.claims")
    return df
    

In [0]:
%python
def claim_vw():
    # Your query or data processing logic here
    df = spark.sql("SELECT * FROM dev.bronze.claims")
    return df

# Call the function and display the result
result_df = claim_vw()
display(result_df)

add column and save it in silver

In [0]:
from pyspark.sql.functions import *

In [0]:
@dlt.table(
    table_properties = {"quality": "silver"},
    comment = "add column",
    name = "claim_silver"
)
def claim_silver():
    df = spark.read.table("LIVE.claim_bronze")
    df = df.withColumn("__insert_date", current_timestamp())
    return df


aggregata on payortype

In [0]:
@dlt.table(
  table_properties = {"quality": "gold"},
  comment = "agg on payor type",
  name = "claim_gold"
)
def claim_gold():
  df = spark.read.table("LIVE.claim_silver")
  df_final = df.groupBy("payorType").agg(count("*").alias("total"))
  return df_final

In [0]:
%sql
select * from dev.etl.claim_gold;